# Exploratory Notebook

- Test Database Utility Functions
- Test API Service
- Explore Results
_____

In [1]:
%cd ../

/Users/tracesmith/Desktop/Trace/Coding/user-recommender


In [2]:
%load_ext autoreload
import os
import pandas as pd

### Test Database (SQLite3)

In [121]:
%autoreload
from recommender.database import utils as db_main
from recommender.database.manager import DatabaseManager

In [19]:
%autoreload
# Populate Data
db_main.ingest_raw_data('dev')

In [4]:
%autoreload
# Check Populated Data
df_check_tags = db_main.read_table('dev', f"select * from course_tags")
df_check_tags.head()

,course_id,course_tags
0,12-principles-animation-toon-boom-harmony-1475,2d-animation
1,2d-racing-game-series-unity-5-1312,game-design
2,2d-racing-games-unity-volume-2-1286,game-art
3,2d-racing-games-unity-volume-2-1286,digital-painting
4,2d-racing-games-unity-volume-2-1286,image-editing


In [122]:
%autoreload
# Create Table for Similarity Results -> matrix factorization
db_main.create_ranking_table_mf('dev','user_ranking_mf', top_n=5)

In [36]:
%autoreload
# Create Table for Similarity Results
db_main.create_ranking_table('dev','user_ranking', top_n=5)

In [30]:
# Sample Data
df_test = pd.DataFrame({'user_handle':['110','110','111','111'],
                        'user_match': ['112','113','157','145'],
                        'similarity': ['80.2','20.8','52.0','48.0']})

In [31]:
%autoreload
# Write Similarty Results to Table
db_main.write_table('dev','test_table',df_test)

In [132]:
%autoreload
# Read from Table
users = '110'
db_main.read_table('dev', f"select * from user_ranking_mf where user_handle = {users}")

In [140]:
t.to_dict(orient='records')

[{'user_handle': '110', 'similar': '5537', 'score': 0.895756185054779},
 {'user_handle': '110', 'similar': '7771', 'score': 0.8567180633544922},
 {'user_handle': '110', 'similar': '5843', 'score': 0.8519828915596008},
 {'user_handle': '110', 'similar': '239', 'score': 0.851238489151001},
 {'user_handle': '110', 'similar': '9530', 'score': 0.8363373279571533},
 {'user_handle': '110', 'similar': '687', 'score': 0.775060772895813},
 {'user_handle': '110', 'similar': '854', 'score': 0.7691600918769836},
 {'user_handle': '110', 'similar': '1199', 'score': 0.7563589811325073},
 {'user_handle': '110', 'similar': '9046', 'score': 0.7409684062004089},
 {'user_handle': '110', 'similar': '7341', 'score': 0.7401623129844666}]

##### Add Index on Results Table (user_ranking)

In [128]:
# Index user handle
with DatabaseManager('dev') as conn:
        try:
            sql_table = f"""CREATE UNIQUE INDEX user_handle_index ON user_ranking_mf (user_handle)"""
            cur = conn.cursor()
            cur.execute(sql_table)
        except Exception as e:
            print(str(e))
            pass

index user_handle_index already exists


### Test Flask API

```
python runserver.py
```


**Python Script**

In [183]:
!export DATABASE_ENV=dev

In [184]:
!export RESULTS_TABLE=user_ranking

In [191]:
%autoreload
os.environ['DATABASE_ENV'] = 'dev'
os.environ['RESULTS_TABLE'] = 'user_ranking_mf'
from server.api import similarity
similarity(user_id='110')

'{"110": [{"similar": "5537", "score": 0.895756185054779}, {"similar": "7771", "score": 0.8567180633544922}, {"similar": "5843", "score": 0.8519828915596008}, {"similar": "239", "score": 0.851238489151001}, {"similar": "9530", "score": 0.8363373279571533}, {"similar": "687", "score": 0.775060772895813}, {"similar": "854", "score": 0.7691600918769836}, {"similar": "1199", "score": 0.7563589811325073}, {"similar": "9046", "score": 0.7409684062004089}, {"similar": "7341", "score": 0.7401623129844666}]}'

**Make post request using CURL from command line**

In [196]:
!curl -X GET -H "Content-type: application/json" -d "{\"user_handle\":\"110\"}" "http://0.0.0.0:5000/api/similarity/"

{"110": [{"similar": "5537", "score": 0.895756185054779}, {"similar": "7771", "score": 0.8567180633544922}, {"similar": "5843", "score": 0.8519828915596008}, {"similar": "239", "score": 0.851238489151001}, {"similar": "9530", "score": 0.8363373279571533}, {"similar": "687", "score": 0.775060772895813}, {"similar": "854", "score": 0.7691600918769836}, {"similar": "1199", "score": 0.7563589811325073}, {"similar": "9046", "score": 0.7409684062004089}, {"similar": "7341", "score": 0.7401623129844666}]}

### Results

In [3]:
# User content
user_assesments = pd.read_csv(os.path.join('data','user_assessment_scores.csv'))
user_interest = pd.read_csv(os.path.join('data','user_interests.csv'))
user_course_views = pd.read_csv(os.path.join('data','user_course_views.csv'))
course_tags = pd.read_csv(os.path.join('data','course_tags.csv'))

In [104]:
# Input User == 2
%autoreload
db_main.read_table('dev', f"select * from user_ranking where user_handle = 2")

,1,2,3,4,5,user_handle
0,"{'user': 4169, 'score': 1.1381}","{'user': 8314, 'score': 1.1228}","{'user': 487, 'score': 1.1176}","{'user': 463, 'score': 1.117}","{'user': 9280, 'score': 1.1147}",2


In [197]:
user_interest[user_interest['user_handle'] == 110]

,user_handle,interest_tag,date_followed
3955,110,angular,2017-06-29 04:29:55
3956,110,javascript-frameworks,2017-06-29 04:29:55
3957,110,javascript-libraries,2017-06-29 04:29:55
3958,110,javascript,2017-06-29 04:29:55
3959,110,c#,2017-06-29 04:29:55
...,...,...,...
4023,110,continuous-integration,2017-06-29 04:29:55
4024,110,performance-monitoring,2017-06-29 04:29:55
4025,110,visual-studio-2010-performance-testing,2017-06-29 04:29:55
4026,110,performance,2017-06-29 04:29:55


In [198]:
user_interest[user_interest['user_handle'] == 5537]

,user_handle,interest_tag,date_followed
167016,5537,angular,2017-07-16 20:15:37
167017,5537,javascript-frameworks,2017-07-16 20:15:37
167018,5537,javascript-libraries,2017-07-16 20:15:37
167019,5537,javascript,2017-07-16 20:15:37
167020,5537,c#,2017-07-16 20:15:37
...,...,...,...
167101,5537,webstorm,2017-07-16 20:15:37
167102,5537,vim,2017-07-16 20:15:37
167103,5537,developer-tools,2017-07-16 20:15:37
167104,5537,domain-driven-design,2017-07-16 20:15:37


In [203]:
user_assesments[user_assesments['user_handle'] == 110]

,user_handle,assessment_tag,user_assessment_date,user_assessment_score


In [204]:
user_assesments[user_assesments['user_handle'] == 5537]

,user_handle,assessment_tag,user_assessment_date,user_assessment_score


In [212]:
user_course_views[user_course_views['user_handle'] == 110].tail(5)

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
2834,110,2017-08-09,spring-boot-first-application,191,Intermediate,1663
2835,110,2017-08-09,spring-data-rest-getting-started,191,Intermediate,5702
2836,110,2017-08-09,spring-jpa-hibernate,131,Intermediate,11153
2837,110,2017-08-10,java-persistence-api-21,68,Intermediate,114
2838,110,2017-08-10,spring-data-rest-getting-started,191,Intermediate,1838


In [210]:
user_course_views[user_course_views['user_handle'] == 5537].tail(5)

,user_handle,view_date,course_id,author_handle,level,view_time_seconds
139808,5537,2017-08-28,tree-based-models-classification,872,Beginner,7011
139809,5537,2017-08-29,building-multi-client-end-to-end-soa-angular,651,Intermediate,44410
139810,5537,2017-08-29,technical-writing-software-documentation,41,Beginner,13642
139811,5537,2017-08-30,advanced-machine-learning-encog,9,Advanced,15076
139812,5537,2017-09-03,advanced-machine-learning-encog-pt2,9,Advanced,13270
